In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession \
    .builder \
    .appName("chemical_loading") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/10/09 18:56:21 WARN Utils: Your hostname, Aymans-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
22/10/09 18:56:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 18:56:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
diapole_moments = spark.read.option("header",True).csv('../data/dipole_moments.csv')
magnet_sheilding_tensors = spark.read.option("header",True).csv('../data/magnetic_shielding_tensors.csv')
mulliken_charges = spark.read.option("header",True).csv('../data/mulliken_charges.csv')
potential_energy = spark.read.option("header",True).csv('../data/potential_energy.csv')
scalar_coupling_contribution = spark.read.option("header",True).csv('../data/scalar_coupling_contributions.csv')
structures = spark.read.option("header",True).csv('../data/structures.csv').withColumnRenamed("x","small_x") \
    .withColumnRenamed("y","small_y")\
    .withColumnRenamed("z","small_z")

In [3]:
dm_pe = diapole_moments.alias('a')\
        .join(potential_energy.alias('b'), diapole_moments.molecule_name == potential_energy.molecule_name, "left")\
        .select("a.molecule_name", "a.X", "a.Y", "a.Z", "b.potential_energy")

In [4]:
dm_pe_mc = dm_pe.alias('a')\
    .join(mulliken_charges.alias('b'), dm_pe.molecule_name == mulliken_charges.molecule_name, "left")\
    .select("a.molecule_name", "a.X", "a.Y", "a.Z", "a.potential_energy", "b.atom_index", "b.mulliken_charge")

In [5]:
dm_pe_mc_mst = dm_pe_mc.alias('a')\
    .join(
        magnet_sheilding_tensors.alias('b'), 
        (dm_pe_mc.molecule_name == magnet_sheilding_tensors.molecule_name) & (dm_pe_mc.atom_index == magnet_sheilding_tensors.atom_index)
        , "left")\
    .select(
        "a.molecule_name", 
        "a.X", 
        "a.Y",
        "a.Z", 
        "a.potential_energy", 
        "a.atom_index", 
        "a.mulliken_charge", 
        "b.XX", "b.YX", "b.ZX", "b.XY", "b.YY", "b.ZY", "b.XZ", "b.YZ", "b.ZZ")

In [6]:
dm_pe_mc_mst_scc = dm_pe_mc_mst.alias('a')\
    .join(
        scalar_coupling_contribution.alias('b'), 
        (dm_pe_mc_mst.molecule_name == scalar_coupling_contribution.molecule_name) & (dm_pe_mc_mst.atom_index == scalar_coupling_contribution.atom_index_0)
        , "left")\
    .select(
        "a.molecule_name",
        "a.X",
        "a.Y",
        "a.Z",
        "a.potential_energy",
        "a.atom_index",
        "a.mulliken_charge",
        "a.XX",
        "a.YX",
        "a.ZX",
        "a.XY",
        "a.YY",
        "a.ZY",
        "a.XZ",
        "a.YZ",
        "a.ZZ",
        "b.atom_index_0",
        "b.atom_index_1",
        "b.type",
        "b.fc",
        "b.sd",
        "b.pso",
        "b.dso"
)

In [7]:
dm_pe_mc_mst_scc_struc = dm_pe_mc_mst_scc.alias('a')\
    .join(
        structures.alias('b'), 
        (dm_pe_mc_mst_scc.molecule_name == structures.molecule_name) & (dm_pe_mc_mst.atom_index == structures.atom_index)
        , "left")\
    .select(
    "a.molecule_name",
    "a.X",
    "a.Y",
    "a.Z",
    "a.potential_energy",
    "a.atom_index",
    "a.mulliken_charge",
    "a.XX",
    "a.YX",
    "a.ZX",
    "a.XY",
    "a.YY",
    "a.ZY",
    "a.XZ",
    "a.YZ",
    "a.ZZ",
    "a.atom_index_0",
    "a.atom_index_1",
    "a.type",
    "a.fc",
    "a.sd",
    "a.pso",
    "a.dso",
    "b.atom",
    "b.small_x",
    "b.small_y",
    "b.small_z"
)

In [8]:
dm_pe_mc_mst_scc_struc.printSchema()

root
 |-- molecule_name: string (nullable = true)
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Z: string (nullable = true)
 |-- potential_energy: string (nullable = true)
 |-- atom_index: string (nullable = true)
 |-- mulliken_charge: string (nullable = true)
 |-- XX: string (nullable = true)
 |-- YX: string (nullable = true)
 |-- ZX: string (nullable = true)
 |-- XY: string (nullable = true)
 |-- YY: string (nullable = true)
 |-- ZY: string (nullable = true)
 |-- XZ: string (nullable = true)
 |-- YZ: string (nullable = true)
 |-- ZZ: string (nullable = true)
 |-- atom_index_0: string (nullable = true)
 |-- atom_index_1: string (nullable = true)
 |-- type: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- sd: string (nullable = true)
 |-- pso: string (nullable = true)
 |-- dso: string (nullable = true)
 |-- atom: string (nullable = true)
 |-- small_x: string (nullable = true)
 |-- small_y: string (nullable = true)
 |-- small_z: string (nullable =

In [9]:
# dm_pe_mc_mst_scc_struc.write.partitionBy("molecule_name").option("header", True).parquet("../data/all_data_parquet")

In [10]:
dm_pe_mc_mst_scc_struc.show()

22/10/09 18:56:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------+-------+-------+-------+-------------------+----------+-------------------+----------+--------+-------+-------+---------+-------+-------+-------+--------+------------+------------+----+-------------------+-------------------+--------------------+--------------------+----+--------------------+-------------------+--------------------+
|   molecule_name|      X|      Y|      Z|   potential_energy|atom_index|    mulliken_charge|        XX|      YX|     ZX|     XY|       YY|     ZY|     XZ|     YZ|      ZZ|atom_index_0|atom_index_1|type|                 fc|                 sd|                 pso|                 dso|atom|             small_x|            small_y|             small_z|
+----------------+-------+-------+-------+-------------------+----------+-------------------+----------+--------+-------+-------+---------+-------+-------+-------+--------+------------+------------+----+-------------------+-------------------+--------------------+--------------------+----+----

In [11]:
df_len = dm_pe_mc_mst_scc_struc.count()

df_len_lst = [i for i in range(df_len)]
dm_pe_mc_mst_scc_struc_pd = dm_pe_mc_mst_scc_struc.to_pandas_on_spark()
dm_pe_mc_mst_scc_struc_pd

/Users/aymansulaiman/miniforge3/envs/spark/lib/python3.10/site-packages/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_0 in memory! (computed 19.1 MiB so far)
22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_3 in memory! (computed 19.0 MiB so far)
22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_2 in memory! (computed 18.9 MiB so far)
22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_5 in memory! (computed 19.1 MiB so far)
22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_4 in memory! (computed 29.8 MiB so far)
22/10/09 18:56:50 WARN MemoryStore: Not enough space to cache rdd_200_7 in memory! (computed 28.6 MiB so far)
22/10/09 18:56:50 WARN BlockManager: Persisting block rdd_200_2 to disk instead.
22/10/09 18:56:50 WARN BlockManager: Persisting block rdd_200_0 to disk instead.
22/10/09 18:56:50 WARN BlockManager: Persisting block rdd_200_7 to disk instead.
22/10/09 18:56:50 WARN BlockManager: Persisting block rdd_200_3 to disk instead.
22/10/09 18:56:5

22/10/09 18:56:56 WARN MemoryStore: Not enough space to cache rdd_200_8 in memory! (computed 43.0 MiB so far)
22/10/09 18:56:56 WARN BlockManager: Persisting block rdd_200_8 to disk instead.
22/10/09 18:56:56 WARN MemoryStore: Not enough space to cache rdd_200_10 in memory! (computed 44.8 MiB so far)
22/10/09 18:56:56 WARN BlockManager: Persisting block rdd_200_10 to disk instead.
22/10/09 18:56:56 WARN MemoryStore: Not enough space to cache rdd_200_13 in memory! (computed 42.4 MiB so far)
22/10/09 18:56:56 WARN BlockManager: Persisting block rdd_200_13 to disk instead.
22/10/09 18:56:56 WARN MemoryStore: Not enough space to cache rdd_200_12 in memory! (computed 42.4 MiB so far)
22/10/09 18:56:56 WARN BlockManager: Persisting block rdd_200_12 to disk instead.
22/10/09 18:56:56 WARN MemoryStore: Not enough space to cache rdd_200_11 in memory! (computed 43.9 MiB so far)
22/10/09 18:56:56 WARN BlockManager: Persisting block rdd_200_11 to disk instead.
22/10/09 18:56:56 WARN MemoryStore: N

22/10/09 18:56:59 WARN MemoryStore: Not enough space to cache rdd_200_0 in memory! (computed 96.9 MiB so far)


,molecule_name,X,Y,Z,potential_energy,atom_index,mulliken_charge,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ,atom_index_0,atom_index_1,type,fc,sd,pso,dso,atom,small_x,small_y,small_z
0,dsgdb9nsd_000001,0.0,0.0,0.0,-40.5236795,4,0.13392300000000001,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.626,28.9013,4,0,1JHC,83.0243,0.254628,1.2585600000000001,0.272012,H,-0.5238136345000001,1.4379326443,0.9063972942
1,dsgdb9nsd_000005,0.0,0.0,-2.8937,-93.4284884,0,-0.049655,0.0482,0.0,0.0,0.0,0.0482,0.0,0.0,0.0,278.2653,None,None,None,None,None,None,None,C,-0.013323931399999999,1.1324657151000002,0.0082758861
2,dsgdb9nsd_000011,2.3605,1.0119,0.0004,-153.8429678,4,0.14323699999999998,29.0037,1.775,-1.4893,-0.2322,28.954,-0.2294,-2.599,-1.6474,30.9785,4,0,1JHC,80.6709,0.17440899999999998,1.0993700000000002,0.5694100000000001,H,-0.8620154031,1.878524808,-0.5647953841
3,dsgdb9nsd_000011,2.3605,1.0119,0.0004,-153.8429678,4,0.14323699999999998,29.0037,1.775,-1.4893,-0.2322,28.954,-0.2294,-2.599,-1.6474,30.9785,4,1,2JHC,-3.7880599999999998,0.16238699999999998,-0.038622000000000004,-0.11675999999999999,H,-0.8620154031,1.878524808,-0.5647953841
4,dsgdb9nsd_000011,2.3605,1.0119,0.0004,-153.8429678,4,0.14323699999999998,29.0037,1.775,-1.4893,-0.2322,28.954,-0.2294,-2.599,-1.6474,30.9785,4,5,2JHH,-17.7483,0.384387,2.43645,-2.65008,H,-0.8620154031,1.878524808,-0.5647953841
5,dsgdb9nsd_000011,2.3605,1.0119,0.0004,-153.8429678,4,0.14323699999999998,29.0037,1.775,-1.4893,-0.2322,28.954,-0.2294,-2.599,-1.6474,30.9785,4,6,3JHH,0.215153,0.084005,0.618686,-0.719418,H,-0.8620154031,1.878524808,-0.5647953841
6,dsgdb9nsd_000013,0.0,0.0597,0.0,-119.1556568,2,-0.357354,186.7049,5.2098,-0.0003,12.0661,171.1007,-0.0014,-0.002,0.0022,161.8507,None,None,None,None,None,None,None,C,0.7216912862,-0.5258335257,-1.2623057014
7,dsgdb9nsd_000016,0.0004,0.0002,0.0,-117.90602879999999,4,0.11145899999999999,32.3577,1.4961,3.5795,1.4959,26.2978,0.8345,8.8492,2.0644,35.7545,4,0,1JHC,104.411,0.34479699999999996,0.796896,0.7253470000000001,H,-0.32275498280000003,2.0268015645,-0.8934783865000001
8,dsgdb9nsd_000016,0.0004,0.0002,0.0,-117.90602879999999,4,0.11145899999999999,32.3577,1.4961,3.5795,1.4959,26.2978,0.8345,8.8492,2.0644,35.7545,4,1,2JHC,-1.0976700000000001,-0.039887,0.39621300000000004,-0.41025900000000004,H,-0.32275498280000003,2.0268015645,-0.8934783865000001
9,dsgdb9nsd_000016,0.0004,0.0002,0.0,-117.90602879999999,4,0.11145899999999999,32.3577,1.4961,3.5795,1.4959,26.2978,0.8345,8.8492,2.0644,35.7545,4,2,2JHC,-1.0938700000000001,-0.040033,0.396431,-0.410369,H,-0.32275498280000003,2.0268015645,-0.8934783865000001


In [13]:
dm_pe_mc_mst_scc_struc_pd['pk'] = df_len_lst

[77.099s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 85.0 (TID 215): Retried waiting for GCLocker too often allocating 2097154 words
22/10/09 18:57:37 WARN TaskMemoryManager: Failed to allocate a page (16777216 bytes), try again.


/Users/aymansulaiman/miniforge3/envs/spark/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/Users/aymansulaiman/miniforge3/envs/spark/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [15]:
dm_pe_mc_mst_scc_struc_pd.to_csv("../data/all_data_csv")

/Users/aymansulaiman/miniforge3/envs/spark/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


22/10/09 18:59:03 WARN TaskSetManager: Stage 129 contains a task of very large size (8888 KiB). The maximum recommended task size is 1000 KiB.


22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_1 in memory! (computed 28.4 MiB so far)
22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_6 in memory! (computed 12.3 MiB so far)
22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_0 in memory! (computed 28.7 MiB so far)
22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_3 in memory! (computed 19.0 MiB so far)
22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_5 in memory! (computed 19.1 MiB so far)
22/10/09 18:59:07 WARN MemoryStore: Not enough space to cache rdd_371_2 in memory! (computed 12.3 MiB so far)
22/10/09 18:59:07 WARN BlockManager: Persisting block rdd_371_5 to disk instead.
22/10/09 18:59:07 WARN BlockManager: Persisting block rdd_371_3 to disk instead.
22/10/09 18:59:07 WARN BlockManager: Persisting block rdd_371_2 to disk instead.
22/10/09 18:59:07 WARN BlockManager: Persisting block rdd_371_0 to disk instead.
22/10/09 18:59:0

22/10/09 18:59:11 WARN MemoryStore: Not enough space to cache rdd_371_7 in memory! (computed 237.1 MiB so far)


22/10/09 18:59:12 WARN MemoryStore: Not enough space to cache rdd_371_12 in memory! (computed 42.4 MiB so far)
22/10/09 18:59:12 WARN MemoryStore: Not enough space to cache rdd_371_10 in memory! (computed 44.8 MiB so far)
22/10/09 18:59:12 WARN MemoryStore: Not enough space to cache rdd_371_13 in memory! (computed 28.3 MiB so far)
22/10/09 18:59:12 WARN MemoryStore: Not enough space to cache rdd_371_8 in memory! (computed 64.4 MiB so far)
22/10/09 18:59:12 WARN MemoryStore: Not enough space to cache rdd_371_11 in memory! (computed 43.9 MiB so far)
22/10/09 18:59:12 WARN BlockManager: Persisting block rdd_371_11 to disk instead.
22/10/09 18:59:12 WARN BlockManager: Persisting block rdd_371_8 to disk instead.
22/10/09 18:59:12 WARN BlockManager: Persisting block rdd_371_10 to disk instead.
22/10/09 18:59:12 WARN BlockManager: Persisting block rdd_371_13 to disk instead.
22/10/09 18:59:12 WARN BlockManager: Persisting block rdd_371_12 to disk instead.
22/10/09 18:59:12 WARN MemoryStore: N

22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_6 in memory! (computed 29.3 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_5 in memory! (computed 28.7 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_0 in memory! (computed 12.3 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_7 in memory! (computed 12.4 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_4 in memory! (computed 12.4 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_1 in memory! (computed 18.9 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_3 in memory! (computed 8.2 MiB so far)
22/10/09 18:59:14 WARN MemoryStore: Not enough space to cache rdd_371_2 in memory! (computed 18.9 MiB so far)


22/10/09 18:59:18 WARN MemoryStore: Not enough space to cache rdd_371_8 in memory! (computed 96.6 MiB so far)
22/10/09 18:59:18 WARN MemoryStore: Not enough space to cache rdd_371_9 in memory! (computed 19.0 MiB so far)
22/10/09 18:59:18 WARN MemoryStore: Not enough space to cache rdd_371_10 in memory! (computed 19.2 MiB so far)
22/10/09 18:59:18 WARN MemoryStore: Not enough space to cache rdd_371_12 in memory! (computed 5.4 MiB so far)
22/10/09 18:59:18 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_371_13 in memory.
22/10/09 18:59:18 WARN MemoryStore: Not enough space to cache rdd_371_13 in memory! (computed 384.0 B so far)


In [12]:
# dm_pe_mc_mst_scc_struc.write.option("header", True).csv("../data/all_data_csv")